# Testing the BYM2 Model: The Penalised-Complexity Priors  

### Goals of this session  
1. Simulate data with different underlying random effects components.  
    - $\delta$ which is pure overdispersion  
    - $S$ which is spatially correlated  
    - A combination of $\delta$ and $S$  
2. Use the BYM2 model to asses how well the model does at caputuring these effects.  
3. simulate a similar set of data this time using covarites.  
4. Asses covariate bias in BYM2, overdispersion, GMRF, and GMRF+Overdispersion model.  
5. Use the ohio dataset as real test case to models  

In [3]:
options(jupyter.plot_mimetypes = 'image/png')
set.seed(123)
rm(list=ls())
library(SpatialEpi)
library(spdep)
library(INLA)
library(RColorBrewer)
library(maptools)
library(maps)
library(ggplot2)
library(sp)
library(lattice)
library(surveillance)
library(parallel)

In [6]:
## Get Pennsylvania map file
load("../homework/hw3/USA_adm2.RData") # From http://gadm.org/

# define the 50 states remove DC
states <- unique(gadm$NAME_1)
states <- as.character(states[!(states %in% c("District of Columbia"))])

# each data unit will be a state with different data
state_data <- lapply(states, function(x) list(name=x))
names(state_data) <- states

In [7]:
apply2 <- function(f, old_key, new_key, state_data, cores=1){
################################################################################
    states <- names(state_data)
    temp <- mclapply(states, function(x) 
        f(state_data[[x]][[old_key]]), mc.cores=cores)
    names(temp) <- states
    for(state in states){
        state_data[[state]][[new_key]] <- temp[[state]]
    }
    state_data
}

In [8]:
state_data <- apply2(function(x) as(gadm[which(gadm$NAME_1==x),], "SpatialPolygons"), 
                     "name", "SpatialPolygons", state_data)

In [9]:
state_data <- apply2(poly2adjmat, "SpatialPolygons", "adjmat", state_data)

In [33]:
delta_simulatar <- function(sp){
    # Given a spatial polygons data set applies
    # a delta to each unit
    sp$ID <- 1:length(sp)
    sp[,paste0("delta_sim_", 1:100)] <- sapply(1:100, function(x) rgamma(length(sp), 4, 4))
    sp
}
                                               
state_data <- apply2(delta_simulatar, "SpatialPolygons","SpatialPolygons2", state_data)

ERROR: Error in .local(x, i, j, ..., value): argument "i" is missing, with no default


In [ ]:
summary(state_data$Californi)